In [1]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
in_porcentajes = r'../data/matrix/porcentaje_estPrecip.csv'
out_path = r'../data/matrix/cuenca_PTPM_CON_M.csv'

In [3]:
porcentajes = pd.read_csv(in_porcentajes, sep=";")#, dtype={'P 80':'int64'}
porcentajes.head(2)

,CANT,CUENCA,P 80,X,Y,porcentaje
0,1,4,26020460,-76.404667,2.487583,0.44543
1,1,4,26020130,-76.418806,2.541833,0.55457


In [4]:
dtparser = lambda x: datetime.strptime(x, '%Y-%m')
indexcols = ['Fecha']
data = pd.read_csv("../data/matrix/PTPM_CON_M.zip",parse_dates = ['Fecha'], date_parser = dtparser,index_col = indexcols)
data.tail()

,Unnamed: 0,CodigoEstacion,Valor
Fecha,,,
2020-01-01,614215,56010030,10.00
2020-02-01,614216,56010030,1.00
2020-03-01,614217,56010030,9.15
2020-04-01,614218,56010030,54.50
2020-05-01,614219,56010030,246.00


In [5]:
data = data['2000':'2019']

In [6]:
data.shape

(417600, 3)

In [7]:
data = data.drop(['Unnamed: 0'], axis=1)
data.head()

,CodigoEstacion,Valor
Fecha,,
2000-01-01,11010010,335.0
2000-02-01,11010010,805.0
2000-03-01,11010010,735.0
2000-04-01,11010010,873.0
2000-05-01,11010010,1149.0


In [8]:
data['Fecha'] = data.index

In [9]:
data2 = data.merge(porcentajes,  left_on="CodigoEstacion", right_on="P 80")


In [10]:
data2.shape

(14880, 9)

In [11]:

def get_table_CUENCA(i):
    data_i = data2[data2.CUENCA==i].reset_index()
    data_i['valor_porcent'] = data_i['Valor']*data_i['porcentaje'] 
    df_i = data_i.pivot(index='Fecha', columns='P 80', values='valor_porcent')
    df_i['PTPM_CON_WA'] = df_i.sum(axis=1)
    df_i['CUENCA'] = i
    return(df_i[['PTPM_CON_WA', 'CUENCA']])


In [12]:
df_all = []
for i in data2.CUENCA.unique():
    df_all.append(get_table_CUENCA(i))
result = pd.concat(df_all) 
result['Fecha'] = result.index

In [13]:
result.to_csv(out_path, index = False, date_format = '%Y-%m')